In [1]:
#Dependencies
import tweepy
import json
import numpy as np
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

In [3]:
#API Keys for twitter
Consumer_APIkey = "ZlxghovIUf8cC7zFcWbAH1wUh"
Consumer_APIsecret = "ulJEQVzIAwn22S90JT6zjtSHQsr8n4hKR1Yz1TlNJJ45TNuSUp"
Access_Token = "772933790498861056-YxzOTJfcJ4m049PgfWG91aSTrjO9xl9"
Access_Tokensecret = "gbAmLW77rzu50fZvSh9iucmO0gW72stIqnLEPwH4qIwUo"

#Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [ ]:
#Target Search Term
target_terms = ['@BBC','@CBS','@CNN','@FoxNews','@nytimes']